# ITNPBD2 Representing and Manipulating Data

# Pandas

## Spreadsheet and Database table like data frames
- Named columns and (if you like) rows
- Selection, aggregation, calculation methods
- Load and save from files

In [ ]:
import pandas as pd

## Read a data frame from a file
- Default delimiter is `,`
- Choice to specify row and column names
- Many more choices we won't worry about yet

In [ ]:
loans=pd.read_csv("data\loans.csv", index_col=0)  # ,index_col=0 to make index first column
display(loans.head())

# Select one or more columns
- Either by `framename.colname` or `framename['colname']`

In [ ]:
ages=loans.Age
display(ages.head())

## Can provide an array of column names
- Note how the index column is also kept

In [ ]:
age_loan=loans[['Age', 'Loan amount']]
display(age_loan.head())

# Select Rows
- By column index

In [ ]:
print(loans.loc[945780])

- By Query

In [ ]:
over60=loans[loans.Age>60]
display(over60.head())

# Selecting rows and columns

In [ ]:
over60Income=loans[loans.Age>60]['Income']
print(over60Income.head())

o60i=loans.loc[loans.Age>60,'Income']
display(o60i.head())

## Create a data frame in your code

In [ ]:
df=pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], index=['x', 'y', 'z'], columns=['a', 'b', 'c'])
print(df)

# Extract Columns by name

In [ ]:
print(df.a)
# or
print(df['a'])

# Extract Rows by Index Name or Location
- Use `loc` to refer to rows by index name
- Use `iloc` to access by row number


In [ ]:
# print(df[1])  # Error - this means a column called 1
print(df.loc['x']) # Correct
print(df.iloc[1]) # Correct

# More sophisticated queries
- Add terms to the search
- Use `query`

In [ ]:
print(loans.loc[(loans.Age> 30) & (loans.Age<50)].head())

print(loans.query("Age > 30 & Age < 50").head())


# A quick look at conditional indexes (masks)

In [ ]:
mask=loans.Age>50
print(mask.head())
print(loans[mask].head())
print(mask.any(),mask.all())

# Describe a dataframe

In [ ]:
print(loans.describe())

## What about the String typed columns?
- List with `unique`
- Summarise with describe()

In [ ]:
print(loans['Own home'].unique())
print(loans['Own home'].describe())

# Aggregation
 - Calculate same thing for each group by value
 - E.g Average income by age

In [ ]:
print(loans.groupby('Gender')['Income'].mean())
print(loans.groupby('Own home')['Own home'].count())

## Here we select only some of the Gender values to aggregate over

In [ ]:
print(loans[loans.Gender.isin(['Male', 'Female', 'M', 'F'])].groupby('Gender')['Income'].count())

## Some coding errors to fix!
- First, count the occurences of each value

In [ ]:
print(loans.groupby('Gender')['Gender'].count())

# Here we are editing and removing certain rows
## Note that `~loans...` means not ...
- The third line of code removes all the rows where the value for gender is in the given list, effectively by selecting every row where the value for gender is NOT in that list

In [ ]:
loans.loc[loans.Gender=='Female']='F'
loans.loc[loans.Gender=='Male']='M'
loans=loans.loc[~loans.Gender.isin(['0', '1', 'D', 'H', 'N'])]
print(loans.groupby('Gender')['Gender'].count())

# Nested Row and Columns Labels
- A `MultiIndex` is hierarchical index
- Consider sales organised by Region and Month
- In some sense, the data are the sales figures and the region and month (though columns) are really identifiers, not data


In [ ]:
sales=pd.read_csv('data\sales.csv',index_col=[0,1])
sales.head()

In [ ]:
sales.loc['USA']

In [ ]:
sales.loc['USA','August']

In [ ]:
sales.groupby(['Region']).mean()

In [ ]:
sales.unstack('Region')

# Relational Tables and Joins
- Relational table design allows a value in one table to refer to an index in another
- Consider a table of customers and orders (simplified)


In [ ]:
customers=pd.read_csv('data\Customers.csv', index_col=0)
orders=pd.read_csv('data\Orders.csv', index_col=0)
display(customers)
display(orders)

# What is the name of the person who bought the drum in order 4?
- Could look it up twice:

In [ ]:
cust_id=orders.loc[4].Cust_ID
print(customers.loc[cust_id].Name)

In [ ]:
# Or all in one line:
print(customers.loc[orders.loc[4].Cust_ID].Name)

## Or perform a join on the whole table

In [ ]:
display(pd.merge(customers, orders, on='Cust_ID'))

# Why not store the data in one table like this anyway?
- Notice that Sandeep hasn't made an order yet - but he is in the customers table
- What if a customer changes their email address? Only one edit needed
- If Sally cancels her order, she vanishes from all records

See https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html for more details

## Outer Joins
 - The above example is an inner join - it only contains data where the key appears in the orders table
- That means Sandeep is missing
- An outer join can be used to include all data from one table, even if it doesn't appear in the other

In [ ]:
display(pd.merge(customers, orders, how='left', on='Cust_ID'))

## In the above,
- `customers` is the left table (first argument)
- `orders` is the right table (second)
- `how='left'` means include all entries in left table (customers) even if they don't appear in the right.
- See the `NaN` entries where no data are found

# Now with Three Tables

In [ ]:
korders=pd.read_csv('data\OrdersWithKey.csv', index_col=0)
products=pd.read_csv('data\Products.csv', index_col=0)
display(products)
display(korders)

In [ ]:
customers.merge(korders, on='Cust_ID').merge(products, on='Product_ID')